# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [61]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [30]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Display sample data
city_data_df.head(10)

City_ID                   City Country      Lat       Lng  Max Temp  \
0        0         Gornopravdinsk      RU  60.0500   69.9000     44.29   
1        1              Morro Bay      US  35.3658 -120.8499     68.40   
2        2           Punta Arenas      CL -53.1500  -70.9167     44.71   
3        3               Thompson      CA  55.7435  -97.8558     28.56   
4        4             Nikolskoye      RU  59.7035   30.7861     48.09   
5        5                Rikitea      PF -23.1203 -134.9692     71.08   
6        6                Lyngdal      NO  58.1376    7.0700     51.89   
7        7  Mayor Pablo Lagerenza      PY -19.9309  -60.7718     82.15   
8        8                  Kapaa      US  22.0752 -159.3190     80.92   
9        9              Carnarvon      AU -24.8667  113.6333     73.38   

   Humidity  Cloudiness  Wind Speed Current Description  
0        86          86        5.91     overcast clouds  
1        88          38        3.36    scattered clouds  
2        70          75       37.98          light rain  
3        86          20        2.30          few clouds  
4        94         100        6.98     overcast clouds  
5        71          23       19.77          light rain  
6        82         100       10.92     overcast clouds  
7        47          71        4.00       broken clouds  
8        70           0        9.22           clear sky  
9        48          30       14.99    scattered clouds

## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [31]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 50



## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [32]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

City_ID            City Country      Lat      Lng  Max Temp  Humidity  \
0         0  Gornopravdinsk      RU  60.0500  69.9000     44.29        86   
2         2    Punta Arenas      CL -53.1500 -70.9167     44.71        70   
3         3        Thompson      CA  55.7435 -97.8558     28.56        86   
4         4      Nikolskoye      RU  59.7035  30.7861     48.09        94   
11       11          Dikson      RU  73.5069  80.5464     30.36        85   
17       17            Hofn      IS  64.2539 -15.2082     39.74        96   
20       20        Betzdorf      DE  50.7833   7.8833     45.18        90   
21       21           Ancud      CL -41.8697 -73.8203     39.79        86   
22       22         Ushuaia      AR -54.8000 -68.3000     46.06        61   
23       23            Lebu      CL -37.6167 -73.6500     46.49        83   

    Cloudiness  Wind Speed   Current Description  
0           86        5.91       overcast clouds  
2           75       37.98            light rain  
3           20        2.30            few clouds  
4          100        6.98       overcast clouds  
11         100       10.67       overcast clouds  
17         100       21.74  heavy intensity rain  
20           2        3.44             clear sky  
21           2        4.00             clear sky  
22          20        8.05            few clouds  
23           0       18.68             clear sky

## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [33]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

City_ID            City Country      Lat      Lng  Max Temp  Humidity  \
0         0  Gornopravdinsk      RU  60.0500  69.9000     44.29        86   
2         2    Punta Arenas      CL -53.1500 -70.9167     44.71        70   
3         3        Thompson      CA  55.7435 -97.8558     28.56        86   
4         4      Nikolskoye      RU  59.7035  30.7861     48.09        94   
11       11          Dikson      RU  73.5069  80.5464     30.36        85   
17       17            Hofn      IS  64.2539 -15.2082     39.74        96   
20       20        Betzdorf      DE  50.7833   7.8833     45.18        90   
21       21           Ancud      CL -41.8697 -73.8203     39.79        86   
22       22         Ushuaia      AR -54.8000 -68.3000     46.06        61   
23       23            Lebu      CL -37.6167 -73.6500     46.49        83   

    Cloudiness  Wind Speed   Current Description  
0           86        5.91       overcast clouds  
2           75       37.98            light rain  
3           20        2.30            few clouds  
4          100        6.98       overcast clouds  
11         100       10.67       overcast clouds  
17         100       21.74  heavy intensity rain  
20           2        3.44             clear sky  
21           2        4.00             clear sky  
22          20        8.05            few clouds  
23           0       18.68             clear sky

## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [34]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()


# Display sample data
hotel_df.head(10)

City Country  Max Temp      Lat      Lng
0   Gornopravdinsk      RU     44.29  60.0500  69.9000
2     Punta Arenas      CL     44.71 -53.1500 -70.9167
3         Thompson      CA     28.56  55.7435 -97.8558
4       Nikolskoye      RU     48.09  59.7035  30.7861
11          Dikson      RU     30.36  73.5069  80.5464
17            Hofn      IS     39.74  64.2539 -15.2082
20        Betzdorf      DE     45.18  50.7833   7.8833
21           Ancud      CL     39.79 -41.8697 -73.8203
22         Ushuaia      AR     46.06 -54.8000 -68.3000
23            Lebu      CL     46.49 -37.6167 -73.6500

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [35]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

City Country  Max Temp      Lat      Lng Hotel Name
0   Gornopravdinsk      RU     44.29  60.0500  69.9000           
2     Punta Arenas      CL     44.71 -53.1500 -70.9167           
3         Thompson      CA     28.56  55.7435 -97.8558           
4       Nikolskoye      RU     48.09  59.7035  30.7861           
11          Dikson      RU     30.36  73.5069  80.5464           
17            Hofn      IS     39.74  64.2539 -15.2082           
20        Betzdorf      DE     45.18  50.7833   7.8833           
21           Ancud      CL     39.79 -41.8697 -73.8203           
22         Ushuaia      AR     46.06 -54.8000 -68.3000           
23            Lebu      CL     46.49 -37.6167 -73.6500

## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [36]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [37]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# Display sample data
hotel_df.head(10)

City Country  Max Temp      Lat      Lng  \
0   Gornopravdinsk      RU     44.29  60.0500  69.9000   
2     Punta Arenas      CL     44.71 -53.1500 -70.9167   
3         Thompson      CA     28.56  55.7435 -97.8558   
4       Nikolskoye      RU     48.09  59.7035  30.7861   
11          Dikson      RU     30.36  73.5069  80.5464   
17            Hofn      IS     39.74  64.2539 -15.2082   
20        Betzdorf      DE     45.18  50.7833   7.8833   
21           Ancud      CL     39.79 -41.8697 -73.8203   
22         Ushuaia      AR     46.06 -54.8000 -68.3000   
23            Lebu      CL     46.49 -37.6167 -73.6500   

                               Hotel Name  
0                                  Hostel  
2                              Hotel Hain  
3                         Burntwood Hotel  
4                 Tourist House - Sablino  
11                                         
17                             Hótel Höfn  
20  Hotel & Restaurant ‚zum weißen Stein‘  
21                     Hotel Arena Gruesa  
22                        Albatross Hotel  
23                       Hostal Las Lilas

## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [39]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]==0]

# Display sample data
clean_hotel_df.head(10)

Empty DataFrame
Columns: [City, Country, Max Temp, Lat, Lng, Hotel Name]
Index: []

**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [40]:
# In case of an empty DataFrame, load the sample data provided
clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
clean_hotel_df.head(10)

City_ID            City Country  Max Temp   Current Description    Lat  \
0        2            Aksu      CN     82.42             clear sky  41.12   
1        7          Touros      BR     80.60         broken clouds  -5.20   
2       10        Morehead      US     75.20             clear sky  37.27   
3       11  Port Elizabeth      ZA     84.20             clear sky -33.92   
4       16         Northam      GB     82.40            few clouds  51.03   
5       18       Hyderabad      IN     87.01                  haze  17.38   
6       20     Port Alfred      ZA     82.00             clear sky -33.59   
7       22          Atuona      PF     79.72             clear sky  -9.80   
8       23           Kapaa      US     73.40  heavy intensity rain  22.08   
9       25      Nikolskoye      RU     88.00             clear sky  59.70   

      Lng                              Hotel Name  
0   80.26                    Pudong Holiday Hotel  
1  -35.46                         INN NEW HORIZON  
2  -87.18                         CCI Express Inn  
3   25.57                  39 On Nile Guest House  
4   -4.22                     Durrant House Hotel  
5   78.47                  Taj Krishna, Hyderabad  
6   26.89                      The Halyards Hotel  
7 -139.03                             Villa Enata  
8 -159.32  Sheraton Kauai Resort at Coconut Beach  
9   30.79                 Tourist House - Sablino

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [41]:
# Set the file name.
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [46]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [48]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Pudong Holiday Hotel</dd>\n<dt>City</dt><dd>Aksu</dd>\n<dt>Country</dt><dd>CN</dd>\n<dt>Current Weather</dt><dd>clear sky and 82.42 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>INN NEW HORIZON</dd>\n<dt>City</dt><dd>Touros</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>broken clouds and 80.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>CCI Express Inn</dd>\n<dt>City</dt><dd>Morehead</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 75.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>39 On Nile Guest House</dd>\n<dt>City</dt><dd>Port Elizabeth</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 84.2 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Durrant House Hotel</dd>\n<dt>City</dt><dd>Northam</dd>\n<dt>Country</dt><dd>GB</dd>\n<dt>Current Weather</dt><dd>few clouds and 82.4 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Taj Krishna, Hyderabad</dd>\n<dt>Cit

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [49]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

Lat     Lng
0  41.12   80.26
1  -5.20  -35.46
2  37.27  -87.18
3 -33.92   25.57
4  51.03   -4.22
5  17.38   78.47
6 -33.59   26.89
7  -9.80 -139.03
8  22.08 -159.32
9  59.70   30.79

## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [60]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))